In [548]:
import jsonlines
import os
import pandas as pd
import numpy as np

In [549]:
def process_method(filename):
    options = {
        "results_spatial_plus_final.jsonl": "spatial+",
        "results_spatial_final.jsonl": "spatial",
        "results_GCNN_zero.jsonl": "gcnn ZERO",
        "results_GCNN_1hidden_layer.jsonl": "gcnn 1HIDDEN",
        "results_GCNN_lin.jsonl": "gcnn LIN",
        "results_GCN.jsonl": "gcnn ReLU",
        "results_GM_Lag.jsonl": "s2sls",
        "results_GM_Error.jsonl": "glmerr",
        "results_Ridge.jsonl": "ols",
        "results_dapsm_final.jsonl": "dapsm",
    }
    return options.get(filename)

In [550]:
def get_meanplusminus(datmean, datstd):
    return (
        datmean.map("{:02.2f}".format)
        + " ± {\footnotesize "
        + datstd.map("{:02.2f}".format)
        + "}"
    )

In [551]:
def get_databin(df):
    datbin_ate_mean = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["ate_se"]
        .mean()
    )
    datbin_ate_std = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["ate_se"]
        .std()
    )
    datbin_ate = get_meanplusminus(datbin_ate_mean, datbin_ate_std)
    datbin_pehe_mean = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["pehe_av"]
        .mean()
    )
    datbin_pehe_std = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["pehe_av"]
        .std()
    )
    datbin_pehe = get_meanplusminus(datbin_pehe_mean, datbin_pehe_std)
    datbin = pd.concat([datbin_ate, datbin_pehe], axis=1)
    return datbin

In [552]:
def get_datcon(df):
    datcon_erf_mean = (
        df[df["treatment_type"] != "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["erf_av"]
        .mean()
    )
    datcon_erf_std = (
        df[df["treatment_type"] != "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["erf_av"]
        .std()
    )
    datcon_erf = get_meanplusminus(datcon_erf_mean, datcon_erf_std)
    datcon_pehe_mean = (
        df[df["treatment_type"] != "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["pehe_av"]
        .mean()
    )
    datcon_pehe_std = (
        df[df["treatment_type"] != "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["pehe_av"]
        .std()
    )
    datcon_pehe = get_meanplusminus(datcon_pehe_mean, datcon_pehe_std)
    datcon = pd.concat([datcon_erf, datcon_pehe], axis=1)
    return datcon

In [553]:
def get_dapsm(df):
    datbin_ate_mean = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["ate_se"]
        .mean()
    )
    datbin_ate_std = (
        df[df["treatment_type"] == "binary"]
        .groupby(["smoothness_binned", "confounding_binned"])["ate_se"]
        .std()
    )
    datbin_ate = get_meanplusminus(datbin_ate_mean, datbin_ate_std)
    return datbin_ate

In [554]:
directory_path = "results/FINAL/"

files_list = []
# Iterate over all the files in the directory
for filename in os.listdir(directory_path):
    # If the filename starts with "results_"
    if filename.startswith("results_"):
        # Construct the full file path
        files_list.append(filename)

In [555]:
def mean_of_list(value):
    if isinstance(value, list):
        return np.mean(value)
    else:
        return value

In [556]:
files_list

['results_spatial_plus_final.jsonl',
 'results_GM_Lag.jsonl',
 'results_GM_Error.jsonl',
 'results_GCN.jsonl',
 'results_GCN_hidden_new.jsonl',
 'results_GCN_first_MT.jsonl',
 'results_GCN_relu_16h.jsonl',
 'results_dapsm_final.jsonl',
 'results_Ridge.jsonl',
 'results_spatial_final.jsonl']

In [557]:
tlb_list = []

for filename in files_list:
    print(f"Filename: {filename}")
    with jsonlines.open(directory_path + "/" + filename) as reader:
        data = [obj for obj in reader]
    df = pd.DataFrame(data)
    df["treatment_type"] = np.where(
        df["envname"].str.contains("disc"), "binary", "continuous"
    )
    # df["smoothness_binned"] = pd.qcut(df["smoothness"], q=2, labels=["low", "high"])
    # df["confounding_binned"] = pd.qcut(df["confounding"], q=2, labels=["low", "high"])

    # Use pd.cut to create the bins
    df["smoothness_binned"] = pd.cut(
        df["smoothness"], bins=[-np.inf, 0.5, np.inf], labels=["low", "high"]
    )
    df["confounding_binned"] = pd.cut(
        df["confounding"], bins=[-np.inf, 0.025, np.inf], labels=["low", "high"]
    )

    df["ate_se"] = df["ate_se"].apply(lambda x: x[0] if isinstance(x, list) else x)

    df["ate_se"] = np.sqrt(df["ate_se"])  # df[""] * 100
    if filename == "results_dapsm_final.jsonl":
        dat = get_dapsm(df).to_frame()
        dat["pehe_bin"] = "n/a"
        dat["erf_av"] = "n/a"
        dat["pehe_con"] = "n/a"
    elif filename == "results_GCN_fin.jsonl":
        continue
    else:
        df["pehe_av"] = np.sqrt(df["pehe_av"])  # df[""] * 100
        df["erf_av"] = np.sqrt(df["erf_av"])  # * 100  df[""]

        datbin = get_databin(df).rename(columns={"pehe_av": "pehe_bin"})
        datcon = get_datcon(df).rename(columns={"pehe_av": "pehe_con"})
        dat = pd.concat([datbin, datcon], axis=1)

    method = process_method(filename)
    dat["method"] = method
    dat.set_index("method", append=True)
    tlb_list.append(dat)

Filename: results_spatial_plus_final.jsonl
Filename: results_GM_Lag.jsonl
Filename: results_GM_Error.jsonl
Filename: results_GCN.jsonl
Filename: results_GCN_hidden_new.jsonl
Filename: results_GCN_first_MT.jsonl
Filename: results_GCN_relu_16h.jsonl
Filename: results_dapsm_final.jsonl
Filename: results_Ridge.jsonl
Filename: results_spatial_final.jsonl


In [558]:
fin = (
    pd.concat(tlb_list)
    .reset_index()
    .rename(
        columns={"smoothness_binned": "smoothness", "confounding_binned": "confounding"}
    )
    .groupby(by=["smoothness", "confounding", "method"])
    .first()
)

In [559]:
fin

ate_se              pehe_bin  \
smoothness confounding method                                                  
low        low         dapsm      0.20 ± {\small 0.13}                   n/a   
                       gcnn ReLU  0.08 ± {\small 0.06}  0.29 ± {\small 0.07}   
                       glmerr     0.04 ± {\small 0.02}  0.13 ± {\small 0.03}   
                       ols        0.05 ± {\small 0.02}  0.13 ± {\small 0.03}   
                       s2sls      0.09 ± {\small 0.12}  0.15 ± {\small 0.08}   
                       spatial    0.05 ± {\small 0.02}  0.13 ± {\small 0.03}   
                       spatial+   0.05 ± {\small 0.02}  0.13 ± {\small 0.04}   
           high        dapsm      0.13 ± {\small 0.13}                   n/a   
                       gcnn ReLU  0.06 ± {\small 0.04}  0.24 ± {\small 0.07}   
                       glmerr     0.06 ± {\small 0.03}  0.14 ± {\small 0.04}   
                       ols        0.06 ± {\small 0.03}  0.14 ± {\small 0.04}   
                       s2sls      0.05 ± {\small 0.04}  0.14 ± {\small 0.04}   
                       spatial    0.06 ± {\small 0.03}  0.14 ± {\small 0.04}   
                       spatial+   0.05 ± {\small 0.03}  0.14 ± {\small 0.04}   
high       low         dapsm      0.10 ± {\small 0.12}                   n/a   
                       gcnn ReLU  0.05 ± {\small 0.04}  0.25 ± {\small 0.06}   
                       glmerr     0.04 ± {\small 0.03}  0.13 ± {\small 0.03}   
                       ols        0.04 ± {\small 0.02}  0.13 ± {\small 0.04}   
                       s2sls      0.06 ± {\small 0.06}  0.13 ± {\small 0.04}   
                       spatial    0.05 ± {\small 0.03}  0.13 ± {\small 0.03}   
                       spatial+   0.04 ± {\small 0.02}  0.13 ± {\small 0.04}   
           high        dapsm      0.21 ± {\small 0.12}                   n/a   
                       gcnn ReLU  0.08 ± {\small 0.05}  0.25 ± {\small 0.07}   
                       glmerr     0.06 ± {\small 0.04}  0.14 ± {\small 0.03}   
                       ols        0.06 ± {\small 0.05}  0.14 ± {\small 0.03}   
                       s2sls      0.03 ± {\small 0.03}  0.15 ± {\small 0.03}   
                       spatial    0.06 ± {\small 0.05}  0.14 ± {\small 0.03}   
                       spatial+   0.06 ± {\small 0.05}  0.14 ± {\small 0.04}   

                                                erf_av              pehe_con  
smoothness confounding method                                                 
low        low         dapsm                       n/a                   n/a  
                       gcnn ReLU  0.57 ± {\small 0.55}  0.75 ± {\small 0.54}  
                       glmerr     0.38 ± {\small 0.37}  0.41 ± {\small 0.37}  
                       ols        0.41 ± {\small 0.40}  0.44 ± {\small 0.39}  
                       s2sls      0.35 ± {\small 0.35}  0.38 ± {\small 0.35}  
                       spatial    0.40 ± {\small 0.39}  0.44 ± {\small 0.39}  
                       spatial+   0.40 ± {\small 0.39}  0.44 ± {\small 0.39}  
           high        dapsm                       n/a                   n/a  
                       gcnn ReLU  0.55 ± {\small 0.37}  0.67 ± {\small 0.36}  
                       glmerr     0.19 ± {\small 0.23}  0.28 ± {\small 0.22}  
                       ols        0.20 ± {\small 0.24}  0.29 ± {\small 0.23}  
                       s2sls      0.23 ± {\small 0.20}  0.27 ± {\small 0.20}  
                       spatial    0.20 ± {\small 0.24}  0.28 ± {\small 0.22}  
                       spatial+   0.20 ± {\small 0.24}  0.28 ± {\small 0.22}  
high       low         dapsm                       n/a                   n/a  
                       gcnn ReLU  0.51 ± {\small 0.52}  0.73 ± {\small 0.50}  
                       glmerr     0.33 ± {\small 0.34}  0.36 ± {\small 0.34}  
                       ols        0.35 ± {\small 0.36}  0.38 ± {\small 0.36}  
                       s2sls      0.34 ± {\small 0.31}  0.37 ± {\sm

In [560]:
with open("leaderboards.txt", "w") as f:
    f.write(
        fin.to_latex(
            index=True,
            escape=False,
            formatters={"name": str.upper},
            float_format="{:.3f}".format,
        )
    )

/var/folders/b_/vlxgm9n10bz569rffzmcc5dc0000gn/T/ipykernel_36057/24736006.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  fin.to_latex(
